# Transformer_from_scratch + Seq2Seq Attention (Blank)
### 실행을 위한 설치

In [ ]:
# 필수: PyTorch (텐서 연산/모델 정의)
!pip install torch

### 이 빈칸 코드로 공부하는 방법

- **학습 목표**: 논문 Eq.(1) / Fig.2 흐름을 코드로 구현하며, 변수명(예: `Q`, `K`, `V`, `attention_logits_QK`)만 보고도 위치를 추적합니다.
- **풀이 방식**: `____`(언더바)로 뚫린 부분만 채운 뒤, 각 연산 직후의 **Shape 주석**이 기대하는 차원과 일치하는지 확인합니다.
- **참고 자료 연결**: 각 코드 블록 바로 위 Markdown에 **치트시트 제목**이 붙어 있고, 코드 위에는 **쿡북 표준 패턴 요약**이 있어 구글링 없이 진행 가능합니다.
- **이 버전의 Attention**: `seq2seq` 스타일의 **additive(MLP) compatibility**로 energy를 계산하는 변형입니다(나머지 Transformer 흐름은 동일).

- 치트시트 섹션 제목은 **해당 코드 블록 바로 위 Markdown 셀**로 배치되어 있습니다.
- 빈칸은 `_`(언더바)로 표시됩니다.

In [ ]:
"""
A from scratch implementation of Transformer network,
following the paper Attention is all you need with a
few minor differences. I tried to make it as clear as
possible to understand and also went through the code
on my youtube channel!
"""

import torch
import torch.nn as nn


## 1) Scaled Dot-Product Attention ↔ `SelfAttention.forward`
## 2) Multi-Head Attention (MHA) ↔ split → attention → concat → projection

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.d_model = embed_size
        self.h = heads
        # [CookBook step.2] d_k = d_model // h ; assert d_k * h == d_model
        self.d_k = self.d_model // ________
        assert (
            ____________ * ____________ == ____________
        ), "Embedding size needs to be divisible by heads"

        # [CookBook step.6] Q = XW^Q, K = XW^K, V = XW^V (Eq.1)
        self.W_V = nn.Linear(__________, ____________)
        self.W_K = nn.Linear(__________, ____________)
        self.W_Q = nn.Linear(__________, ____________)
        # [CookBook step.12] W^O output projection after Concat(heads) (Fig.2)
        self.W_O = nn.Linear(__________, ____________)

        # [Seq2Seq Attention Pattern] energy = ReLU(W_a([Q ; K])) ; attention = softmax(energy)
        # (This mirrors seq2seq_attention.py: energy = relu(self.energy(cat(h_repeat, enc_states))))
        self.W_a = nn.Linear(__________ * self.d_k, 1)
        self.relu = nn.ReLU()

    def forward(self, values, keys, query, mask):
        # Get number of training examples
        N = ____________

        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # [CookBook step.6] Linear projections to Q, K, V
        V = self.W_V(values)  # (N, value_len, d_model)
        K = self.W_K(keys)  # (N, key_len, d_model)
        Q = self.W_Q(query)  # (N, query_len, d_model)

        # [CookBook step.7] Split into heads: (N, L, d_model) -> (N, L, h, d_k)
        V = V.reshape(N, value_len, ____________, ____________)  # (N, value_len, h, d_k)
        K = K.reshape(N, key_len, ____________, ____________)  # (N, key_len, h, d_k)
        Q = Q.reshape(N, query_len, ____________, ____________)  # (N, query_len, h, d_k)

        # [Seq2Seq Attention Pattern] repeat/expand query over key positions to score each (q,k)
        Q_expanded = Q.unsqueeze(____)
        # (N, query_len, h, 1, d_k)
        Q_expanded = Q_expanded.expand(N, query_len, self.h, key_len, self.d_k)
        # (N, query_len, h, key_len, d_k)

        K_expanded = K.permute(0, ____, ____, ____)
        # (N, h, key_len, d_k)
        K_expanded = K_expanded.unsqueeze(1)
        # (N, 1, h, key_len, d_k)
        K_expanded = K_expanded.expand(N, query_len, self.h, key_len, self.d_k)
        # (N, query_len, h, key_len, d_k)

        QK_concat = torch.cat((Q_expanded, K_expanded), dim=____)
        # (N, query_len, h, key_len, 2*d_k)

        # [Seq2Seq Attention Pattern] energy -> attention weights (softmax over key_len)
        additive_energy = self.W_a(QK_concat)
        # (N, query_len, h, key_len, 1)
        additive_energy = self.relu(additive_energy)
        # (N, query_len, h, key_len, 1)
        attention_logits_additive = additive_energy.squeeze(____)
        # (N, query_len, h, key_len)

        attention_logits_additive = attention_logits_additive.permute(0, ____, ____, ____)
        # (N, h, query_len, key_len)

        # [CookBook step.9] mask == 0 -> -inf
        if mask is not None:
            attention_logits_additive = attention_logits_additive.masked_fill(__________ == 0, float("-1e20"))
            # (N, h, query_len, key_len)

        attention_weights = torch.softmax(attention_logits_additive, dim=____)
        # (N, h, query_len, key_len)

        # [CookBook step.11] out = sum_k attention(q,k) * V(k)
        out_heads = torch.einsum("____________________", [attention_weights, V])
        # (N, query_len, h, d_k)
        out = out_heads.reshape(N, query_len, ____________ * ____________)
        # (N, query_len, d_model)

        # [CookBook step.12] Concat(heads) W^O
        out = self.W_O(out)
        # (N, query_len, d_model)

        return out


## 5) Add & Norm + FFN (Eq.2) ↔ `TransformerBlock.forward`

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        # [CookBook step.14] FFN(x)=max(0, xW1+b1)W2+b2 (Eq.2)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        # [CookBook step.13] Multi-Head Attention sub-layer
        multihead_attention_output = self.attention(value, key, query, mask)
        # (N, query_len, d_model)

        # [CookBook step.13] Add & Norm: LayerNorm(x + Sublayer(x))
        attention_residual_add = ____________ + ____________
        # (N, query_len, d_model)
        post_attention_layernorm = self.norm1(attention_residual_add)
        # (N, query_len, d_model)
        x = self.dropout(post_attention_layernorm)
        # (N, query_len, d_model)

        # [CookBook step.14] FFN in explicit steps (so shapes are visible)
        ffn_linear1_output = self.feed_forward[0](x)
        # (N, query_len, forward_expansion*d_model)
        ffn_relu_output = self.feed_forward[1](ffn_linear1_output)
        # (N, query_len, forward_expansion*d_model)
        ffn_linear2_output = self.feed_forward[2](ffn_relu_output)
        # (N, query_len, d_model)

        # [CookBook step.14] Add & Norm after FFN
        ffn_residual_add = ____________ + ____________
        # (N, query_len, d_model)
        post_ffn_layernorm = self.norm2(ffn_residual_add)
        # (N, query_len, d_model)
        out = self.dropout(post_ffn_layernorm)
        # (N, query_len, d_model)

        return out


## 6) Embedding + Positional Encoding ↔ `Encoder/Decoder.forward`

In [ ]:
class Encoder(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        embed_size,
        num_layers,
        heads,
        device,
        forward_expansion,
        dropout,
        max_length,
    ):

        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
                for _ in range(num_layers)
            ]
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, src_token_ids, src_padding_mask):
        # [CookBook step.1] src_token_ids: (N, src_len)
        N, src_len = src_token_ids.shape

        # [CookBook step.5] positions = arange(0, L).expand(N, L)
        position_ids = torch.arange(0, ____________).to(self.device)
        # (src_len,)
        positions = position_ids.expand(__________, ____________)
        # (N, src_len)

        # [CookBook step.4~5] token embedding + position embedding
        token_embedding_d_model = self.word_embedding(src_token_ids)
        # (N, src_len, d_model)
        positional_embedding_d_model = self.position_embedding(positions)
        # (N, src_len, d_model)
        out = self.dropout(____________ + ____________)
        # (N, src_len, d_model)

        # [CookBook step.15] Encoder stack: out = layer(out, out, out, src_mask)
        for layer in self.layers:
            out = layer(____________, ____________, ____________, ____________)
            # (N, src_len, d_model)

        return out


## 3) Encoder / Decoder의 3가지 Attention 매핑

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.norm = nn.LayerNorm(embed_size)
        self.attention = SelfAttention(embed_size, heads=heads)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)

    #   - Decoder Masked Self-Attention
    #   - Encoder-Decoder Attention (Cross-Attention)
    def forward(self, x, value, key, src_mask, trg_mask):
        # [CookBook step.16] Masked self-attention (future tokens blocked by trg_mask)
        masked_self_attention_output = self.attention(____________, ____________, ____________, ____________)
        # (N, trg_len, d_model)

        # [CookBook step.16] Add & Norm
        residual_add = ____________ + ____________
        # (N, trg_len, d_model)
        query = self.dropout(self.norm(residual_add))
        # (N, trg_len, d_model)

        # [CookBook step.17] Cross-attention + FFN via TransformerBlock(value=enc_out, key=enc_out, query=query)
        out = self.transformer_block(____________, ____________, ____________, ____________)
        # (N, trg_len, d_model)

        return out


## 6) Embedding + Positional Encoding ↔ `Encoder/Decoder.forward`
## 7) Output projection (Logits) ↔ `Decoder.fc_out`

In [ ]:
class Decoder(nn.Module):
    def __init__(
        self,
        trg_vocab_size,
        embed_size,
        num_layers,
        heads,
        forward_expansion,
        dropout,
        device,
        max_length,
    ):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
                for _ in range(num_layers)
            ]
        )
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, trg_token_ids, enc_out, src_padding_mask, trg_causal_mask):
        # [CookBook step.1] trg_token_ids: (N, trg_len)
        N, trg_len = trg_token_ids.shape

        # [CookBook step.5] positions = arange(0, L).expand(N, L)
        position_ids = torch.arange(0, ____________).to(self.device)
        # (trg_len,)
        positions = position_ids.expand(__________, ____________)
        # (N, trg_len)

        # [CookBook step.4~5] token embedding + position embedding
        token_embedding_d_model = self.word_embedding(trg_token_ids)
        # (N, trg_len, d_model)
        positional_embedding_d_model = self.position_embedding(positions)
        # (N, trg_len, d_model)
        x = self.dropout(____________ + ____________)
        # (N, trg_len, d_model)

        # [CookBook step.18] Decoder stack
        for layer in self.layers:
            x = layer(____________, ____________, ____________, ____________, ____________)
            # (N, trg_len, d_model)

        # [CookBook step.18] vocab projection -> logits
        out = self.fc_out(____________)
        # (N, trg_len, trg_vocab_size)

        return out


## 4) Mask(마스크) ↔ `make_src_mask`, `make_trg_mask`

In [ ]:
class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        trg_pad_idx,
        embed_size=512,
        num_layers=6,
        forward_expansion=4,
        heads=8,
        dropout=0,
        device="cpu",
        max_length=100,
    ):

        super(Transformer, self).__init__()

        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length,
        )

        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length,
        )

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src_token_ids):
        # [CookBook step.3] src_padding_mask = (src != pad).unsqueeze(1).unsqueeze(2)
        src_is_not_pad = (src_token_ids != ____________)
        # (N, src_len)
        src_padding_mask = src_is_not_pad.unsqueeze(____)
        # (N, 1, src_len)
        src_padding_mask = src_padding_mask.unsqueeze(____)
        # (N, 1, 1, src_len)
        return src_padding_mask.to(self.device)

    def make_trg_mask(self, trg_token_ids):
        # [CookBook step.3] trg_causal_mask = tril(ones(L,L)).expand(N, 1, L, L)
        N, trg_len = trg_token_ids.shape
        trg_ones = torch.ones((__________, ____________))
        # (trg_len, trg_len)
        trg_lower_triangular = torch.tril(trg_ones)
        # (trg_len, trg_len)
        trg_causal_mask = trg_lower_triangular.expand(__________, 1, ____________, ____________)
        # (N, 1, trg_len, trg_len)

        return trg_causal_mask.to(self.device)

    def forward(self, src_token_ids, trg_token_ids):
        # [CookBook step.19] src_mask -> encoder -> decoder
        src_padding_mask = self.make_src_mask(____________)
        # (N, 1, 1, src_len)
        trg_causal_mask = self.make_trg_mask(____________)
        # (N, 1, trg_len, trg_len)

        enc_src = self.encoder(____________, ____________)
        # (N, src_len, d_model)
        out = self.decoder(____________, ____________, ____________, ____________)
        # (N, trg_len, trg_vocab_size)

        return out


In [ ]:
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    x = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2, 0], [1, 8, 7, 3, 4, 5, 6, 7, 2]]).to(
        device
    )
    # (N, src_len)
    trg = torch.tensor([[1, 7, 4, 3, 5, 9, 2, 0], [1, 5, 6, 2, 4, 7, 6, 2]]).to(device)
    # (N, trg_len)

    src_pad_idx = 0
    trg_pad_idx = 0
    src_vocab_size = 10
    trg_vocab_size = 10
    model = Transformer(
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        trg_pad_idx,
        device=device,
    ).to(device)

    trg_input_ids = trg[:, :-1]
    # (N, trg_len-1)
    out = model(x, trg_input_ids)
    # (N, trg_len-1, trg_vocab_size)
    print(out.shape)
